# 加密货币指数计算演示

本演示将逐步展示如何构建和计算加密货币市值加权指数。

> **注意**：运行前请先参考 [`../README.md`](../README.md) 配置环境和导入必要模块。

## 市值加权指数算法原理

### 什么是市值加权指数？

市值加权指数（Market Capitalization Weighted Index）是一种常见的指数编制方法，广泛应用于股票市场和加密货币市场。在这种方法中，每个成分资产在指数中的权重由其市值大小决定。

### 核心概念

**市值（Market Capitalization）**：指某个资产的总市场价值，计算公式为：
- 市值 = 当前价格 × 流通供应量

**权重（Weight）**：指某个资产在指数中所占的比例，计算公式为：
- 权重 = 该资产市值 / 所有成分资产市值总和

### 算法基本流程

1. **确定成分资产**：选择要纳入指数的资产（如前10大、前20大等）
2. **计算各资产市值**：获取每个资产的当前价格和流通供应量
3. **计算权重**：根据市值占比确定每个资产的权重
4. **设定基准值**：选择一个基准日期和基准指数值（通常为1000或100）
5. **计算指数值**：根据权重和价格变动计算当前指数值

### 指数计算公式

**基准日指数值**：
```
指数值 = 基准值（如1000）
```

**后续日期指数值**：
```
指数值 = 基准值 × (当前总市值 / 基准日总市值)
```

其中，总市值是所有成分资产市值的加权平均。

### 市值加权的优势

1. **客观性**：权重完全基于市场价值，无需主观判断
2. **代表性**：大市值资产对指数影响更大，更能反映市场整体状况
3. **可投资性**：投资者可以按权重比例构建投资组合
4. **广泛认可**：标普500、纳斯达克等主要指数都采用此方法

### 市值加权的局限性

1. **集中度风险**：大市值资产过度影响指数表现
2. **价格偏差**：可能放大泡沫或高估资产的影响
3. **波动性**：大市值资产的剧烈波动会显著影响指数

### 在加密货币中的应用

加密货币市值加权指数通常会：
- 排除稳定币（如USDT、USDC）
- 排除包装币和衍生品（如WBTC、stETH）
- 只包含原生代币
- 定期调整成分币种以反映市场变化

## 步骤0：必不可少的一步————环境设置

在获取数据之前，需要进行如下环境设置。

In [ ]:
# 环境设置
import sys
from pathlib import Path
import logging

# 设置静默模式
logging.basicConfig(level=logging.WARNING)
for logger_name in ['src', 'BatchDownloader', 'src.downloaders.daily_aggregator']:
    logging.getLogger(logger_name).setLevel(logging.WARNING)

# 项目路径设置
project_root = Path.cwd().parent.parent  
sys.path.insert(0, str(project_root))
print(f"✅ 项目路径: {project_root}")

# 导入模块
from src.classification.unified_classifier import UnifiedClassifier
from src.downloaders.daily_aggregator import DailyDataAggregator

# 指定使用数据库（更短的文件名）
working_db_path = project_root / 'data' / 'market.db'
print(f"🔧 使用数据库: {working_db_path.name} ({working_db_path.stat().st_size / 1024 / 1024:.1f}MB)")

# 创建实例，启用数据库模式并指定数据库路径
aggregator = DailyDataAggregator(use_database=True, db_path=str(working_db_path))
classifier = UnifiedClassifier()

print("🎉 环境设置完成！数据库模式已启用")

## 步骤1：获取代币数据

现在我们开始获取和处理代币数据。我们会使用项目中的数据聚合器来获取指定日期的市场数据。

In [ ]:
# 重新导入修复排名的显示工具
from importlib import reload
import src.utils.display_utils
reload(src.utils.display_utils)
from src.utils.display_utils import CryptoDataDisplayer

# 数据获取
date_str = '2024-12-21'  # 使用最新日期
coin_num = 5  # 设置要获取的代币数量
raw_data = aggregator.get_daily_data(date_str)
print(f"📅 数据日期:", date_str)
print(f"📊 原始数据: {len(raw_data)} 个代币")

# 数据处理和展示
displayer = CryptoDataDisplayer()
crypto_data = displayer.clean_data(raw_data)

print(f"\n🎯 清理后有效数据: {len(crypto_data)} 个代币")

# 显示前10大市值币种，检查排名是否连续
displayer.show_table(crypto_data, 
                    top_n=coin_num,
                    title= f"前{coin_num}大市值币种")

## 步骤2：计算指数权重

在获得筛选后的代币数据后，我们需要计算每个代币在指数中的权重。权重基于市值占比，市值越大的代币在指数中的权重越高。

In [ ]:
# 选择前N大市值币种作为指数成分
top_coins = crypto_data.head(coin_num).copy()

# 计算权重
total_market_cap = top_coins['market_cap'].sum()
top_coins['weight'] = top_coins['market_cap'] / total_market_cap

print(f"🎯 指数成分币种: {len(top_coins)} 个")
print(f"💰 总市值: ${total_market_cap:,.0f}")
print(f"📊 权重分布:")

# 显示权重分布
weight_display = top_coins[['symbol', 'name', 'market_cap', 'weight']].head(coin_num).copy()
weight_display['权重(%)'] = (weight_display['weight'] * 100).round(2)
weight_display['市值($)'] = weight_display['market_cap'].apply(lambda x: f"{x:,.0f}")

print(f"\n🔝 前{coin_num}大权重币种:")
for _, row in weight_display.iterrows():
    print(f"  {row['symbol']:>8s} | {row['权重(%)']:>6.2f}% | ${row['市值($)']:>15s}")

# 验证权重总和
weight_sum = top_coins['weight'].sum()
print(f"\n✅ 权重验证: {weight_sum:.6f} (应该接近1.0)")

## 步骤3：设定基准指数值

为了计算指数，我们需要设定一个基准日期和基准指数值。通常会选择一个具有代表性的日期作为基准日，并设定基准指数值为1000（这是金融市场的常见做法）。

在这个演示中，我们将当前日期作为基准日，指数基准值设为1000。

In [ ]:
# 设定基准参数
base_date = date_str  # 基准日期
base_index_value = 1000  # 基准指数值
base_total_market_cap = total_market_cap  # 基准日总市值

print(f"📅 基准日期: {base_date}")
print(f"📊 基准指数值: {base_index_value}")
print(f"💰 基准总市值: ${base_total_market_cap:,.0f}")

# 创建指数基本信息
index_info = {
    'name': f'Crypto{coin_num} 市值加权指数',
    'base_date': base_date,
    'base_value': base_index_value,
    'base_market_cap': base_total_market_cap,
    'constituents_count': len(top_coins),
    'constituents': top_coins[['symbol', 'name', 'weight']].to_dict('records')
}

print(f"\n🎯 指数信息:")
print(f"  指数名称: {index_info['name']}")
print(f"  成分币种: {index_info['constituents_count']} 个")
print(f"  基准日期: {index_info['base_date']}")
print(f"  基准指数值: {index_info['base_value']}")

# 当前日期的指数值（由于是基准日，所以就是基准值）
current_index_value = base_index_value
print(f"\n📈 当前指数值: {current_index_value}")
print(f"📈 指数涨跌: +0.00% (基准日)")

# 保存指数构成详情
print(f"\n💡 指数构成 (权重 > 1%的币种):")
significant_weights = top_coins[top_coins['weight'] > 0.01]  # 权重大于1%
for _, coin in significant_weights.iterrows():
    print(f"  {coin['symbol']:>8s} | {coin['weight']*100:>6.2f}% | {coin['name']}")

## 步骤4：模拟不同日期的指数计算

为了演示指数是如何随时间变化的，我们可以模拟使用不同日期的数据来计算指数值。指数计算的基本公式是：

**指数值 = 基准值 × (当前总市值 / 基准总市值)**

让我们尝试获取其他日期的数据并计算指数变化。

In [ ]:
# 获取可用的日期列表进行演示
from pathlib import Path
daily_files_path = Path(project_root) / 'data' / 'daily' / 'daily_files'
available_dates = []

print(f"🔍 扫描数据目录: {daily_files_path}")

if daily_files_path.exists():
    # 遍历年份目录
    for year_dir in sorted(daily_files_path.glob('20*')):
        if year_dir.is_dir():
            # 遍历月份目录
            for month_dir in sorted(year_dir.glob('*')):
                if month_dir.is_dir():
                    # 遍历日期文件
                    for date_file in sorted(month_dir.glob('*.csv')):
                        date_str = date_file.stem  # 获取不带扩展名的文件名
                        if len(date_str) == 10 and date_str.count('-') == 2:  # 格式验证 YYYY-MM-DD
                            available_dates.append(date_str)

available_dates.sort(reverse=True)  # 最新日期在前
print(f"📅 可用日期总数: {len(available_dates)} 个")

if len(available_dates) > 0:
    print(f"📅 最近2个日期: {available_dates[:2]}")
    
    # 选择几个不同日期进行指数计算演示
    demo_dates = available_dates[:2] if len(available_dates) >= 5 else available_dates
    print(f"\n🎯 用于演示的日期: {demo_dates}")

    # 计算多日指数
    index_history = []

    for demo_date in demo_dates:
        try:
            # 获取该日期数据
            date_data = aggregator.get_daily_data(demo_date)
            if len(date_data) == 0:
                print(f"⚠️  {demo_date}: 无数据")
                continue
                
            # 清理数据
            clean_date_data = displayer.clean_data(date_data)
            
            # 确保我们有足够的数据
            if len(clean_date_data) < coin_num:
                print(f"⚠️  {demo_date}: 数据不足({len(clean_date_data)}个)，跳过")
                continue
                
            # 获取相同的成分币种（按symbol匹配）
            base_symbols = set(top_coins['symbol'].tolist())
            
            # 计算当日这些币种的总市值
            current_constituents = clean_date_data[clean_date_data['symbol'].isin(base_symbols)]
            
            if len(current_constituents) < len(base_symbols) * 0.8:  # 至少80%的币种有数据
                print(f"⚠️  {demo_date}: 成分币种数据不完整({len(current_constituents)}/{len(base_symbols)})，跳过")
                continue
                
            current_total_market_cap = current_constituents['market_cap'].sum()
            
            # 计算指数值
            index_value = base_index_value * (current_total_market_cap / base_total_market_cap)
            
            # 计算变化率
            change_rate = (index_value - base_index_value) / base_index_value * 100
            
            index_history.append({
                'date': demo_date,
                'index_value': index_value,
                'change_rate': change_rate,
                'market_cap': current_total_market_cap,
                'available_coins': len(current_constituents)
            })
            
            print(f"✅ {demo_date}: 指数值 {index_value:.2f}, 变化 {change_rate:+.2f}%")
            
        except Exception as e:
            print(f"❌ {demo_date}: 计算失败 - {str(e)}")
            continue

    # 显示指数历史
    if index_history:
        print(f"\n📊 {index_info['name']} 历史表现:")
        print(f"{'日期':>12s} | {'指数值':>10s} | {'涨跌幅':>10s} | {'成分币种':>8s}")
        print("-" * 50)

        for record in sorted(index_history, key=lambda x: x['date'], reverse=True):
            change_color = "📈" if record['change_rate'] >= 0 else "📉"
            print(f"{record['date']:>12s} | {record['index_value']:>10.2f} | {change_color}{record['change_rate']:>8.2f}% | {record['available_coins']:>6d}/30")

        # 计算统计信息
        index_values = [r['index_value'] for r in index_history]
        max_value = max(index_values)
        min_value = min(index_values)
        latest_value = index_history[0]['index_value']  # 按日期排序后的第一个
        
        print(f"\n📊 指数统计 (基于{len(index_history)}个交易日):")
        print(f"  最新指数值: {latest_value:.2f}")
        print(f"  期间最高值: {max_value:.2f}")
        print(f"  期间最低值: {min_value:.2f}")
        print(f"  期间波动范围: {((max_value - min_value) / base_index_value * 100):.2f}%")
        
        # 计算期间收益率
        oldest_value = sorted(index_history, key=lambda x: x['date'])[0]['index_value']
        period_return = (latest_value - oldest_value) / oldest_value * 100
        print(f"  期间总收益率: {period_return:+.2f}%")
    else:
        print("⚠️  无法计算指数历史，可能是数据不完整")
else:
    print("❌ 没有找到任何可用的历史数据文件")

print(f"\n✅ 指数计算演示完成！")

## 总结

通过以上演示，我们完整展示了加密货币市值加权指数的计算过程：

### 关键步骤回顾

1. **数据获取与清理**：从历史数据中获取代币的价格和市值信息
2. **权重计算**：基于市值计算每个代币在指数中的权重
3. **基准设定**：确定基准日期和基准指数值
4. **指数计算**：使用公式计算不同日期的指数值

### 核心公式

```
指数值 = 基准值 × (当前总市值 / 基准总市值)
权重 = 该币种市值 / 所有成分币种总市值
```

### 实际应用价值

这种指数计算方法可以用于：
- **投资组合基准**：衡量投资表现
- **市场分析**：了解整体市场走势  
- **风险管理**：分散投资风险
- **产品开发**：创建指数基金或ETF

### 下一步

在实际应用中，您可能需要考虑：
- 定期调整成分币种（如季度或月度）
- 处理新币上市和退市的情况
- 考虑流动性和交易量因素
- 实现实时指数更新机制

这个演示为您提供了构建更复杂加密货币指数产品的基础框架。